### Developing a neural network model to predict if a customer stays or leaves a bank.

In [8]:
# Importing the dataset
dataset = read.csv('Churn_Modelling.csv')
dataset = dataset[4:14]

In [9]:
head(dataset)

CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,France,Female,42,2,0.00,1,1,1,101348.88,1
608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
502,France,Female,42,8,159660.80,3,1,0,113931.57,1
699,France,Female,39,1,0.00,2,0,0,93826.63,0
850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1


In [10]:
# Encoding the categorical variables as factors
dataset$Geography = as.numeric(factor(dataset$Geography,
                                      levels = c('France', 'Spain', 'Germany'),
                                      labels = c(1, 2, 3)))
dataset$Gender = as.numeric(factor(dataset$Gender,
                                   levels = c('Female', 'Male'),
                                   labels = c(1, 2)))

In [11]:
# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(dataset$Exited, SplitRatio = 0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

In [12]:
# Feature Scaling
training_set[-11] = scale(training_set[-11])
test_set[-11] = scale(test_set[-11])

In [14]:
# Fitting ANN to the Training set
# install.packages('h2o')
library(h2o)
h2o.init(nthreads = -1)
model = h2o.deeplearning(y = 'Exited',
                         training_frame = as.h2o(training_set),
                         activation = 'Rectifier',
                         hidden = c(5,5),
                         epochs = 100,
                         train_samples_per_iteration = -2)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    C:\Users\pavan\AppData\Local\Temp\RtmpMru9aE/h2o_pavan_started_from_r.out
    C:\Users\pavan\AppData\Local\Temp\RtmpMru9aE/h2o_pavan_started_from_r.err


Starting H2O JVM and connecting: ............................................................ 
[1] "localhost"
[1] 54321
[1] TRUE
[1] -1
[1] "Failed to connect to localhost port 54321: Connection refused"


Warning message:
"running command 'curl 'http://localhost:54321'' had status 1"

[1] 1


ERROR: Error in h2o.init(nthreads = -1): H2O failed to start, stopping execution.


In [ ]:
# Predicting the Test set results
y_pred = h2o.predict(model, newdata = as.h2o(test_set[-11]))
y_pred = (y_pred > 0.5)
y_pred = as.vector(y_pred)

In [ ]:
# Making the Confusion Matrix
cm = table(test_set[, 11], y_pred)

h2o.shutdown()

In [ ]:
cm